- Look into using the `Dataset` object for the datasets [helpful link](https://www.tensorflow.org/guide/data).

In [ ]:
import tensorflow as tf

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

from tensorflow.keras import layers

import cv2
import numpy as np
import random
import matplotlib.pyplot as plt

from data_sequence import DataSequence


MODELS = ["forward", "left", "right"]  # Also represents the class names.
MODEL_I = 0  # Model index, determines which model to train, hence which data to use, (chosen from MODELS).


# The folder where the data is stored. This is here to easily switch between local and Google Colab env.
PATH_TO_DATA_DIR = "data"
# PATH_TO_DATA_DIR = "drive/My Drive/Auto_RC_Car/data"  # For when using Google Colab.


VALIDATION_SPLIT = 0.2
TEST_SPLIT = 0  # Zero for no test set. Note: Only applicable when custom batches are used.
# For testing, I simply test it on the road physically for a more accurate test.


USE_DATA_AUG = False
BATCH_SIZE = 256  # If using custom batches, needs to be even because half will be original data, half augmented data.
NUM_CLASSES = 3

REMOVE_F_SAMPLES = True  # Whether to remove forward samples or not (to balance the data).
F_SAMPLES_TO_REMOVE = 4000


USE_ROAD_MASK = True  # Whether to excract the road mask for each frame and use as an additional input.
USE_ROAD_CDIST = True  # Whether to excract the central road distance (mid bottom->mid top) to use as an additional input.


CROP_HEIGHT = True  # Whether to crop the image from top to HEIGHT_CROP pixels.
HEIGHT_TO_CROP = 95  # How many pixels to crop from the top.
RESIZE_IMG = True  # Whether to resize the images or not to the value below, IMG_SHAPE.
IMG_SHAPE = (200, 100)  # (w, h)
USE_YUV = True  # Whether to use YUV colour space, otherwise, RGB.
BLUR_IMG = True  # Whether to blur the images as part of the pre-processing.




USE_CUSTOM_BATCHES = False  # Whether to use batches that have the same ratio of class samples.
# Ratios of classes in each batch. This can be done because there are significantly more forward samples (staying in lane). However, I am still
# experimenting with using custom batches.
FORWARD_RATIO = 0.5
LEFT_RATIO    = 0.25
RIGHT_RATIO   = 0.25



# Data augmentation parameters
ROT_RANGE = 4
BRIGHT_MIN = 0.6
BRIGHT_MAX = 1.32
HORI_FLIP = False




SEED_NUM = 6
tf.random.set_seed(SEED_NUM)
np.random.seed(SEED_NUM)
random.seed(SEED_NUM)





data_seq = DataSequence(BATCH_SIZE, NUM_CLASSES, USE_DATA_AUG, SEED_NUM, USE_CUSTOM_BATCHES)

data_seq.load_data(path_to_data_dir=PATH_TO_DATA_DIR, model_to_load=MODELS[MODEL_I])
data_seq.pre_process_data(CROP_HEIGHT, HEIGHT_TO_CROP, RESIZE_IMG, IMG_SHAPE, USE_YUV, BLUR_IMG)
data_seq.split_into_classes(REMOVE_F_SAMPLES, F_SAMPLES_TO_REMOVE)
data_seq.split_val_test(VALIDATION_SPLIT, TEST_SPLIT, FORWARD_RATIO, LEFT_RATIO, RIGHT_RATIO)


if USE_DATA_AUG:
    data_seq.create_aug_gen(ROT_RANGE, BRIGHT_MIN, BRIGHT_MAX, HORI_FLIP)
    
if USE_ROAD_MASK:
    data_seq.excract_road_masks()
    
if USE_ROAD_CDIST:
    data_seq.excract_road_masks()  # The masks are still needed to get the distances.
    data_seq.excract_centre_distance()



# Just getting a small sample of the training data to visualise.
x_sample_batch, y_sample_batch = data_seq.get_batch(32)
print("\nTemp samples:", x_sample_batch.shape, y_sample_batch.shape)

# Visualising the data 

In [ ]:
# For plotting images in a grid.
def plot_imgs(images, labels, rows=3, cols=3, fig_w=15, fig_h=10):
    fig, axis = plt.subplots(rows, cols, figsize=(fig_w, fig_h))
    fig.tight_layout()

    sample_index = 0
    for row in range(rows):
        for col in range(cols):
            if sample_index >= images.shape[0]:
                break
            img = images[sample_index]            
            label = labels[sample_index]
            
            sample_index += 1

            ax = axis[row, col]
            ax.set_title(str(label))
            ax.imshow(img)
            #ax.axis("off")

In [ ]:
plot_imgs(x_sample_batch, y_sample_batch)

###  Samples per class

In [ ]:
# Note: when using custom batches, this does not matter because each batch will have equal ratio of samples as set above.
if not USE_CUSTOM_BATCHES:  # If using custom batches, the number of class samples per batch will have a fixed ratio.
    a_dictionary = {"Forward": data_seq.forward_x.shape[0], "Left": data_seq.left_x.shape[0], "Right": data_seq.right_x.shape[0]}
    keys = a_dictionary.keys()
    values = a_dictionary.values()

    plt.bar(keys, values) 

# Testing data augmentation 

In [ ]:
if USE_DATA_AUG:
    data_flow = data_seq.datagen.flow(x_sample_batch, y_sample_batch, batch_size=32)
    data_flow_batch = data_flow.next()
    plot_imgs(data_flow_batch[0], data_flow_batch[1])

# Visualising road masks

In [ ]:
plot_imgs(data_seq.x_train_masks, data_seq.y_train)

# Visualising road centre distances 

In [ ]:

sample_imgs_idx = np.random.choice(data_seq.x_train.shape[0], 32)
sample_imgs = data_seq.x_train[sample_imgs_idx]
sample_imgs_dist = data_seq.x_train_dist[sample_imgs_idx] * IMG_SHAPE[1]

for i in range(32):
    # For drawing a line to represent the distance.
    point1 = (IMG_SHAPE[0]//2, IMG_SHAPE[1])
    point2 = (IMG_SHAPE[0]//2, int(IMG_SHAPE[1] - sample_imgs_dist[i]))

    sample_imgs[i] = cv2.line(sample_imgs[i], point1, point2, (0, 255, 0), thickness=2)
    

plot_imgs(sample_imgs, sample_imgs_dist)  # Passing in sample_imgs_dist as the "labels" so distance is displayed on title.

# Creating the model

In [ ]:

# First model for the frames.
input1_frames = tf.keras.Input(shape=(IMG_SHAPE[1], IMG_SHAPE[0], 3))

in1_x = layers.Conv2D(24, 5, padding="same", strides=2, activation="elu")(input1_frames)
in1_x = layers.Conv2D(36, 5, strides=2, activation="elu")(in1_x)

in1_x = layers.Conv2D(48, 5, padding="same", strides=2, activation="elu")(in1_x)
in1_x = layers.Conv2D(64, 3, activation="elu")(in1_x)

in1_x = layers.Conv2D(64, 3, padding="same", activation="elu")(in1_x)
in1_x = layers.MaxPooling2D(2)(in1_x)

in1_x = layers.Dropout(0.5)(in1_x)


fcl = layers.Flatten()(in1_x)  # Fully connected layer(s) (fcl)


# Second model for the road masks.
if USE_ROAD_MASK:
    input2_masks = tf.keras.Input(shape=(IMG_SHAPE[1], IMG_SHAPE[0], 1))
    in2_x = layers.Conv2D(16, 5, strides=2, activation="elu")(input2_masks)
    in2_x = layers.MaxPooling2D(4, padding="same", strides=2)(in2_x)
    in2_x = layers.Conv2D(28, 3, padding="same", strides=2, activation="elu")(in2_x)
    in2_x = layers.MaxPooling2D(3, padding="same", strides=1)(in2_x)
    in2_x = layers.Conv2D(48, 3, strides=1, activation="elu")(in2_x)
    in2_x = layers.MaxPooling2D(2)(in2_x)
    in2_x = layers.Flatten()(in2_x)

    fcl = layers.concatenate([fcl, in2_x])

    
fcl = layers.Dense(100, activation="elu")(fcl)
fcl = layers.Dropout(0.5)(fcl)

fcl = layers.Dense(50, activation="elu")(fcl)
fcl = layers.Dropout(0.5)(fcl)

fcl = layers.Dense(10, activation="elu")(fcl)

# For the road centre distances.
if USE_ROAD_CDIST:
    input3_distances = tf.keras.Input(shape=(1, ))
    fcl = layers.concatenate([fcl, input3_distances])

fcl = layers.Dropout(0.5)(fcl)


outputs = layers.Dense(3, activation="softmax")(fcl)


if USE_ROAD_MASK and USE_ROAD_CDIST:
    model = tf.keras.Model(inputs=[input1_frames, input2_masks, input3_distances], outputs=outputs)
elif USE_ROAD_MASK and not USE_ROAD_CDIST:
    model = tf.keras.Model(inputs=[input1_frames, input2_masks], outputs=outputs)
elif USE_ROAD_CDIST and not USE_ROAD_MASK:
    model = tf.keras.Model(inputs=[input1_frames, input3_distances], outputs=outputs)
else:
    model = tf.keras.Model(inputs=input1_frames, outputs=outputs)

optimizer = tf.keras.optimizers.Adam(lr=0.000015)

# Maybe loss needs  to be categorical_crossentropy.
model.compile(loss="mse", optimizer=optimizer, metrics=["accuracy"])
model.summary()

# Training the model

In [ ]:
def scheduler(epoch, lr):
    if epoch <= 1000:
        return 0.001
    elif epoch <= 1600:
        return 0.0001
    elif epoch <= 2200:
        return 0.00005
    else:
        return 0.00003
    

callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir='./logs')
#     tf.keras.callbacks.LearningRateScheduler(scheduler)
]



# Calculating the class weights if class weights were to be used.
y_labels = []
for label in data_seq.y_train:
    if (label==np.array([1, 0, 0])).all(): y_labels.append(0)
    elif (label==np.array([0, 1, 0])).all(): y_labels.append(1)
    elif (label==np.array([0, 0, 1])).all(): y_labels.append(2)

y_labels = np.array(y_labels)
print(y_labels.shape)

from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight("balanced", np.unique(y_labels), y_labels)
print("Class weights:", class_weights)

class_weights = dict(enumerate(class_weights))
print("Class weights:", class_weights)

In [ ]:
EPOCHS = 1000
MODEL_NAME = "8_2_1_combined_noDAug_lowLR_fromStart"  # The name the model will be saved in.
SAVE_MODEL = True


if not USE_CUSTOM_BATCHES:
    
    if USE_DATA_AUG:
        history = model.fit(data_seq.datagen.flow(data_seq.x_train, data_seq.y_train, batch_size=BATCH_SIZE),
                            steps_per_epoch=data_seq.x_train.shape[0]//BATCH_SIZE,
                            validation_data=(data_seq.x_val, data_seq.y_val),
                            epochs=EPOCHS,
                            callbacks=callbacks)
    else:
        if not USE_ROAD_MASK and not USE_ROAD_CDIST:
            history = model.fit(data_seq.x_train, data_seq.y_train, batch_size=BATCH_SIZE,
                                steps_per_epoch=data_seq.x_train.shape[0]//BATCH_SIZE,
                                validation_data=(data_seq.x_val, data_seq.y_val),
                                epochs=EPOCHS,
                                callbacks=callbacks)
            
        elif USE_ROAD_MASK and not USE_ROAD_CDIST:
            history = model.fit([data_seq.x_train, data_seq.x_train_masks], data_seq.y_train, batch_size=BATCH_SIZE,
                                steps_per_epoch=data_seq.x_train.shape[0]//BATCH_SIZE,
                                validation_data=([data_seq.x_val, data_seq.x_val_masks], data_seq.y_val),
                                epochs=EPOCHS,
                                callbacks=callbacks)
            
        elif USE_ROAD_CDIST and not USE_ROAD_MASK:
            history = model.fit([data_seq.x_train, data_seq.x_train_dist], data_seq.y_train, batch_size=BATCH_SIZE,
                                steps_per_epoch=data_seq.x_train.shape[0]//BATCH_SIZE,
                                validation_data=([data_seq.x_val, data_seq.x_val_dist], data_seq.y_val),
                                epochs=EPOCHS,
                                callbacks=callbacks)
            
        elif USE_ROAD_MASK and USE_ROAD_CDIST:
            history = model.fit([data_seq.x_train, data_seq.x_train_masks, data_seq.x_train_dist],
                                data_seq.y_train, batch_size=BATCH_SIZE,
                                steps_per_epoch=data_seq.x_train.shape[0]//BATCH_SIZE,
                                validation_data=([data_seq.x_val, data_seq.x_val_masks, data_seq.x_val_dist], data_seq.y_val),
                                epochs=EPOCHS,
                                callbacks=callbacks)

    
else:
    history = model.fit(data_seq,
                        steps_per_epoch = data_seq.train_n_samples // BATCH_SIZE,
                        validation_data = (data_seq.x_val, data_seq.y_val),
                        validation_steps = data_seq.x_val.shape[0] // BATCH_SIZE,
                        epochs = EPOCHS,
                        callbacks=callbacks,
                        shuffle=False)

# Saving the trained model and its history.
if SAVE_MODEL:
    model.save(f"saved_models/model_{MODELS[MODEL_I][:1]}_{MODEL_NAME}.h5")
    
    history_array = np.array([history.history["loss"],
                              history.history["accuracy"],
                              history.history["val_loss"],
                              history.history["val_accuracy"]])
    with open(f"saved_models/history_{MODELS[MODEL_I][:1]}_{MODEL_NAME}.npy", "wb") as file:
        np.save(file, history_array)


In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.legend(["training", "validation"])
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss plot")

plt.subplot(1, 2, 2)
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.legend(["training", "validation"])
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Accuracy plot")

# Testing individual data augmentation

### Rotation 

In [ ]:
datagen_rot = tf.keras.preprocessing.image.ImageDataGenerator(
    fill_mode="constant",
    rotation_range=ROT_RANGE,
)

data_flow = datagen_rot.flow(x_sample_batch, y_sample_batch, batch_size=32)
plot_imgs(data_flow.next()[0], data_flow.next()[1])

### Brightness

In [ ]:
datagen_bright = tf.keras.preprocessing.image.ImageDataGenerator(
    brightness_range=(BRIGHT_MIN, BRIGHT_MAX),
    rescale=1./255
)

data_flow = datagen_bright.flow(x_sample_batch, y_sample_batch, batch_size=32)
plot_imgs(data_flow.next()[0], data_flow.next()[1])

###  Horizontal Flipping

In [ ]:
datagen_flip = tf.keras.preprocessing.image.ImageDataGenerator(
    horizontal_flip=HORI_FLIP,
)

data_flow = datagen_flip.flow(x_sample_batch, y_sample_batch, batch_size=32)
plot_imgs(data_flow.next()[0], data_flow.next()[1])

###  Zoom

In [ ]:
datagen_zoom = tf.keras.preprocessing.image.ImageDataGenerator(
    fill_mode="constant",
    zoom_range=[0.5, 1]
)

data_flow = datagen_zoom.flow(x_sample_batch, y_sample_batch, batch_size=32)
plot_imgs(data_flow.next()[0], data_flow.next()[1])